In [72]:
import requests
from datetime import timedelta, date, datetime
import json
import pandas as pd
import os
import csv

In [73]:
def daterange(start_date):
    end_date = datetime.now() - timedelta(days=1)
    current_date = datetime(start_date.year, start_date.month, start_date.day)
    one_day = timedelta(days=1)
    while current_date <= end_date:
        if current_date.month not in [1, 2, 11, 12]:
            yield current_date
        current_date += one_day

In [74]:
def get_daily_game_ids(api_key, year, month, day):
    ''' Fetches game IDs from the daily summary endpoint. '''
    try:
        # Ensure month and day are integers and format them to two digits
        month_str = f"{int(month):02d}"
        day_str = f"{int(day):02d}"
    except ValueError:
        raise ValueError("Month and Day must be convertible to integers.")

    # Construct the URL based on provided parameters
    url = f"https://api.sportradar.com/mlb/trial/v7/en/games/{year}/{month_str}/{day_str}/summary.json"
    
    # Set headers to define expected response format
    headers = {'Accept': 'application/json'}
    
    # Send the API request
    response = requests.get(url, params={'api_key': api_key}, headers=headers)
    
    # Debug print to check URL and response status
    print(f"Request URL: {url} | Status Code: {response.status_code}")

    if response.status_code == 200:
        data = response.json()
        games = data.get('league', {}).get('games', [])
        print(f"Games found on {year}-{month_str}-{day_str}: {len(games)}")
        return [game['game']['id'] for game in games]
    elif response.status_code >= 500:
        print("Server error, consider retrying the request")
        return []
    else:
        print(f"Failed to retrieve data: Status Code {response.status_code}")
        return []

In [89]:
def extract_pitchers(team_data, team_type, pitchers, custom_game_id):
    for player in team_data.get('players', []):
        # Ensure that 'statistics' and 'pitching' keys are present
        stats = player.get('statistics', {})
        pitching_stats = stats.get('pitching', {})
        
        earned_run_avg = pitching_stats.get('seasontd', {}).get('era', '0')
        
        
        if 'overall' in pitching_stats:
            pitcher_name = f"{player.get('preferred_name', '')} {player.get('last_name', '')}"
            ip = pitching_stats['overall'].get('ip_2', 'N/A')  # Innings pitched
            h = pitching_stats['overall']['onbase'].get('h', 'N/A')  # Hits
            r = pitching_stats['overall']['runs'].get('total', 'N/A')  # Runs
            er = pitching_stats['overall']['runs'].get('earned', 'N/A')  # Earned runs
            bb = pitching_stats['overall']['onbase'].get('bb', 'N/A')  # Walks
            so = pitching_stats['overall']['outs'].get('ktotal', 'N/A')  # Strikeouts
            hr = pitching_stats['overall']['onbase'].get('hr', 'N/A')  # Home runs
            batters_faced = pitching_stats['overall'].get('bf', 'N/A')  # Batters faced
            pitches = pitching_stats['overall'].get('pitches', {}).get('count', '0')  # Total pitches
            strikes_total = pitching_stats['overall'].get('pitches', {}).get('ktotal', '0')  # Total pitches
            
            strikes_contact = 'N/A'
            strikes_swinging = pitching_stats['overall'].get('outcome', {}).get('kswing', '0')# Swinging strikes
            strikes_looking = pitching_stats['overall'].get('outcome', {}).get('klook', '0')
            inplay_gb_total = pitching_stats['overall'].get('in_play', {}).get('groundball', '0')  # Ground balls
            inplay_fb_total = pitching_stats['overall'].get('in_play', {}).get('flyball', 'N/A')  # Fly balls
            inplay_ld = pitching_stats['overall'].get('in_play', {}).get('linedrive', 'N/A')  # Line drives
            inplay_unk = '0'  # Unknown inplay
            game_score = 
            inherited_runners =
            inherited_score =
            wpa_def = 
            leverage_index_avg = 
            re24_def = 

            pitchers.append({
                'home_away': team_type,
                'pitcher_name': pitcher_name,
                'ip': ip,
                'h': h,
                'r': r,
                'er': er,
                'bb': bb,
                'so': so,
                'hr': hr,
                'earned_run_avg': earned_run_avg,
                'batters_faced': batters_faced,
                'pitches': pitches,
                'strikes_total': strikes_total,
                'strikes_contact': strikes_contact,
                'strikes_swinging': strikes_swinging,
                'strikes_looking': strikes_looking,
                'inplay_gb_total': inplay_gb_total,
                'inplay_fb_total': inplay_fb_total,
                'inplay_ld': inplay_ld,
                'inplay_unk': inplay_unk,
                'game_id': custom_game_id
            })

    return pitchers

In [93]:
def fetch_and_store_pitchers(api_key, start_date):
    pitchers = []  # Initialize the list outside the for loop to maintain scope throughout the function

    for current_date in daterange(start_date):
        year = current_date.year
        month = current_date.month
        day = current_date.day
        game_ids = get_daily_game_ids(api_key, year, month, day)
        for game_id in game_ids[:1]:  # Limiting to the first game for testing or example purposes
            print(game_id)
            url = f'https://api.sportradar.com/mlb/trial/v7/en/games/{game_id}/extended_summary.json?api_key={api_key}'
            response = requests.get(url)

            if response.status_code == 200:
                data = response.json()
                game_info = data['game']
                home_abbr = game_info['home']['abbr'] if 'home' in game_info else 'unknown'
                away_abbr = game_info['away']['abbr'] if 'away' in game_info else 'unknown'
                date = datetime.strptime(game_info['scheduled'], "%Y-%m-%dT%H:%M:%S%z").strftime("%Y%m%d")
                day_night = game_info['day_night']
                custom_game_id = f"{home_abbr}{away_abbr}{date}{day_night}"

                extract_pitchers(data['game']['home'], 'home', pitchers, custom_game_id)
                extract_pitchers(data['game']['away'], 'away', pitchers, custom_game_id)
            else:
                print(f"Failed to fetch data: {response.status_code} {response.text}")

        break
    # Writing to CSV
    with open('pitchers-test.csv', 'w', newline='') as csvfile:
        fieldnames = [
            'home_away',
            'pitcher_name',
            'ip',
            'h',
            'r',
            'er',
            'bb',
            'so',
            'hr',
            'earned_run_avg',
            'batters_faced',
            'pitches',
            'strikes_total',
            'strikes_contact',
            'strikes_swinging',
            'strikes_looking',
            'inplay_gb_total',
            #'inplay_fb_total',
            #'inplay_ld',
            #'inplay_unk',
            'game_id'
        ]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for pitcher in pitchers:
            writer.writerow(pitcher)

        print("Pitching data written to 'pitchers-test.csv'")

In [94]:
api_key = '95w69RTFvT2czN4vnvoLnW8Os95JWg4SBFjJF1f0'  # Replace with your actual API key
start_date = datetime(2024, 5, 5).date()
fetch_and_store_pitchers(api_key, start_date)

Request URL: https://api.sportradar.com/mlb/trial/v7/en/games/2024/05/05/summary.json | Status Code: 200
Games found on 2024-05-05: 15
d853b5e4-7124-47c4-a43a-f3f9ea994aac
Pitching data written to 'pitchers-test.csv'
